<a href="https://colab.research.google.com/github/ykitaguchi77/Laboratory_course/blob/master/12.%20web%E3%82%A2%E3%83%97%E3%83%AA%E3%82%92%E4%BD%9C%E3%81%A3%E3%81%A6%E3%81%BF%E3%82%88%E3%81%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Streamlit test app**

In [1]:
# prompt: gdrive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 1. 必要なライブラリのインストール
!pip install streamlit --q
!pip install pyngrok --q
!pip install streamlit-option-menu --q

# 2. ngrokのセットアップ
from pyngrok import ngrok

# ngrokのAuthtoken設定 (初回のみ必要)
ngrok.set_auth_token('ngrok api code here')

# 3. サンプルのStreamlitアプリケーション作成


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 87.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 97.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 829.3/829.3 kB 37.0 MB/s eta 0:00:00


In [3]:
import shutil
shutil.copy("/content/drive/MyDrive/AI_laboratory_course/classification.pth", "/content" )

'/content/classification.pth'

In [4]:
import streamlit as st
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import io

# モデルの定義
class FruitClassifier(nn.Module):
    def __init__(self):
        super(FruitClassifier, self).__init__()
        # ResNet18をベースに使用
        self.resnet = models.resnet18(pretrained=False)
        # 最終層を2クラス分類用に変更
        num_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_features, 2)

    def forward(self, x):
        return self.resnet(x)

def load_model():
    model = FruitClassifier()
    # strict=Falseを追加して、完全一致でなくても読み込めるようにする
    state_dict = torch.load('/content/classification.pth', map_location=torch.device('cpu'))
    model.load_state_dict(state_dict)
    model.eval()
    return model

def preprocess_image(image):
    # 学習時と同じ前処理を適用
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    image = preprocess(image)
    return image.unsqueeze(0)

def main():
    st.title('🍎 リンゴ・イチゴ 判別アプリ 🍓')
    st.write('画像をアップロードして、リンゴかイチゴかを判定します')

    # モデルのロード
    try:
        model = load_model()
        classes = ["apple", "strawberry"]

        # ファイルアップローダー
        uploaded_file = st.file_uploader("画像をアップロードしてください", type=['png', 'jpg', 'jpeg'])

        if uploaded_file is not None:
            # 画像の表示
            image = Image.open(uploaded_file)
            st.image(image, caption='アップロードされた画像', use_column_width=True)

            # 予測
            try:
                # 画像の前処理
                input_tensor = preprocess_image(image)

                # 推論
                with torch.no_grad():
                    output = model(input_tensor)
                    probabilities = torch.nn.functional.softmax(output[0], dim=0)
                    predicted_class = torch.argmax(probabilities).item()

                # 結果の表示
                st.write('## 判定結果')
                st.write(f'この画像は「**{classes[predicted_class]}**」です')

                # 確率の表示
                st.write('### 確率')
                for i, prob in enumerate(probabilities):
                    st.write(f'{classes[i]}: {prob.item()*100:.2f}%')
                    # プログレスバーで視覚化
                    st.progress(prob.item())

            except Exception as e:
                st.error('画像の処理中にエラーが発生しました。')
                st.write(f'エラー詳細: {str(e)}')

    except Exception as e:
        st.error('モデルの読み込み中にエラーが発生しました。')
        st.write(f'エラー詳細: {str(e)}')

if __name__ == '__main__':
    main()

2025-01-27 07:54:20.422 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-27 07:54:20.568 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-01-27 07:54:20.573 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-27 07:54:20.575 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-27 07:54:20.579 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-27 07:54:20.582 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-27 07:54:20.583 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
/usr/local/lib/python3.11/dist-packages/torchvi

In [5]:
# 4. Streamlitアプリの起動とngrokトンネルの作成
!streamlit run app.py &>/dev/null&

In [29]:
# 新しい接続を作成
# バインドアドレスを明示的に指定
public_url = ngrok.connect(addr="127.0.0.1:8501", proto="http")
print(public_url)


NgrokTunnel: "https://9599-34-23-119-185.ngrok-free.app" -> "http://127.0.0.1:8501"
